In [37]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from torch.utils.data import TensorDataset, DataLoader
from tqdm.notebook import trange, tqdm

import pickle
import random

rng = np.random.default_rng(926334)
np.set_printoptions(suppress=True)

In [16]:
ROOT = 'C:/Users/Matt/Documents/Python Scripts/SnowComp/realtime/'
MODIS_CNN_PATH = ROOT + 'model_32_18_8_3_0.13_50_1399_0.0001' #'modis_model'

import sys
sys.path.append(ROOT)

import torch
import captum.attr as attrib
from modis_cnn import Net as ModisNet

net_kwargs = {'cdim1': 32,
              'cdim2': 18,
              'cdim3': 8,
              'kernel_sz': 3,
              'dropout': 0.13,
              'ldim': 50}


modis_net = ModisNet(**net_kwargs)
modis_net.load_state_dict(torch.load(MODIS_CNN_PATH))

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
modis_net.to(device)

modis_net.train()
print("model is in Training mode")

c1 dim: 19
mp0 dim: 17
c2 dim: 15
mp1 dim: 13
c3 dim: 11
mp2 dim: 9
flattened_dim 648


Net(
  (conv1): Conv2d(2, 32, kernel_size=(3, 3), stride=(1, 1))
  (avgpool): AvgPool2d(kernel_size=3, stride=1, padding=0)
  (conv2): Conv2d(32, 18, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(18, 8, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=648, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=1, bias=True)
  (dropout1): Dropout2d(p=0.13, inplace=False)
  (dropout2): Dropout2d(p=0.26, inplace=False)
  (BatchNorm1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (BatchNorm3): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [5]:
data_path = "C:/Users/Matt/Dropbox/SnowComp/FinalData/"

y_train  = pd.read_csv(data_path + "train_y.csv")
y_test  = pd.read_csv(data_path + "test_y.csv")

x_train = np.load(data_path + "train_dataset.npy")
x_test = np.load(data_path + "test_datast.npy")

## Make dropout predictions and estimations of uncertainty

In [35]:
#@title Get Modis CNN predictions
def dout_predict(cnn, x): 
    x = torch.Tensor(x).type(torch.FloatTensor)
    
    cnn.train()    
    vals_sub = []
    
    x = TensorDataset(x)
    sub_loader = DataLoader(x, batch_size=5000)
    with torch.no_grad():
        for images in sub_loader:
            images = images[0].to(device)
            vals_sub.append(cnn(images).cpu().numpy())
            
    vals_sub = np.concatenate(vals_sub, axis = 0)    
    
    return vals_sub.squeeze()

In [38]:
preds = np.zeros((x_train.shape[0], 50))
for i in trange(50):
    preds_train[:,i] = dout_predict(modis_net, x_train)
np.save("C:/Users/Matt/Dropbox/SnowComp/FinalData/DropoutPredsTrain.npy", preds_train)

  0%|          | 0/50 [00:00<?, ?it/s]

In [42]:
preds_test = np.zeros((x_test.shape[0], 50))
for i in trange(50):
    preds_test[:,i] = dout_predict(modis_net, x_test)
np.save("C:/Users/Matt/Dropbox/SnowComp/FinalData/DropoutPredsTest.npy", preds_test)

  0%|          | 0/50 [00:00<?, ?it/s]